In [1]:
import pandas
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
pm = u"\u00B1"

In [26]:
def process_data_RODE(csv_path, alg_name, fxn=np.median, last_n=10):
    df = pandas.read_csv(csv_path)
    if alg_name == "qmix":
        df = df.iloc[::4, :]
    cols = [c for c in df.columns if 'MIN' not in c and 'MAX' not in c]
    win_rate = [n for n in cols if n != 'Step']
    
    results = []
    errors = []
    for seed in win_rate:
        print(seed)
        df_filt = df[['Step', seed]].dropna()
        max_step = df_filt[['Step']].max()['Step']
        print("Max step before: ", max_step)
        if alg_name == 'mappo' or alg_name == 'ippo':
            if not max_step >= 4.7e6:
                errors.append((alg_name, seed, max_step))
        elif alg_name == 'qmix':
            if not max_step >= 1.7e6:
                errors.append((alg_name, seed, max_step))
        if max_step < 4e6:
            print("Cut at 2e6")
            max_step = 2e6
        elif max_step < 9e6:
            print("Cut at 5e6")
            max_step = 5e6
        else:
            print("Cut at 10e6")
            max_step = 10e6
        print("Max step after: ", max_step)
        df_final = df_filt.loc[(df_filt['Step'] >= max_step-1e6) & (df_filt['Step'] <= max_step)]
        val = df_final.tail(last_n)[seed].median()
        if val is np.nan:
            print("ERROR IN SEED, ", seed, "CSV: ", csv_path)
            errors.append((alg_name, seed))
            results.append(val)
        else:
            results.append(val)
    results = np.array(results)
    print(results)
    res, std = fxn(results), np.std(results)
    print(res)
    if len(errors) > 0:
        print("ERROR!")
    return df_final, res, std, results, errors

In [4]:
def process_data(csv_path, alg_name, fxn=np.median, num_steps=1e6, num_top=10):
    df = pandas.read_csv(csv_path)
    if alg_name == "qmix":
        df = df.iloc[::4, :]
    cols = [c for c in df.columns if 'MIN' not in c and 'MAX' not in c]
    win_rate = [n for n in cols if n != 'Step']
    
    results = []
    errors = []
    for seed in win_rate:
        print(seed)
        df_filt = df[['Step', seed]].dropna()
        max_step = df_filt[['Step']].max()['Step']
        print("Max step before: ", max_step)
        if alg_name == 'mappo' or alg_name == 'ippo':
            if not max_step >= 4.7e6:
                errors.append((alg_name, seed, max_step))
        elif alg_name == 'qmix':
            if not max_step >= 1.7e6:
                errors.append((alg_name, seed, max_step))
        if max_step < 4e6:
            print("Cut at 2e6")
            max_step = 2e6
        elif max_step < 9e6:
            print("Cut at 5e6")
            max_step = 5e6
        else:
            print("Cut at 10e6")
            max_step = 10e6
        print("Max step after: ", max_step)
        df_final = df_filt.loc[(df_filt['Step'] >= max_step-1e6) & (df_filt['Step'] <= max_step)]
        val = df_final.nlargest(num_top, seed)[seed].mean()
        if val is np.nan:
            print("error in seed: ", seed)
        else:
            results.append(val)
        results.append(val)
    results = np.array(results)
    mean, std = fxn(results), np.std(results)
    if len(errors) > 0:
        print("ERROR!")
    return df, mean, std, results, errors

In [5]:
# def process_data(csv_path, num_steps=1e6, num_top=2):
#     df = pandas.read_csv(csv_path)
#     cols = [c for c in df.columns if 'MIN' not in c and 'MAX' not in c]
#     win_rate = [n for n in cols if n != 'Step']
#     df_filt = df[cols]
#     df_temp = df_filt
#     df_filt["Median"] = df_filt[win_rate].median(axis=1, skipna=False)
    
#     df_final = df_filt[["Step", "Median"]].dropna()
#     max_step = df_final[['Step']].max()['Step']
#     print(max_step)
#     if max_step < 5e6:
#         max_step = 2e6
#     elif max_step < 10e6:
#         max_step = 5e6
#     else:
#         max_step = 10e6
#     print(max_step)
#     df_final = df_final.loc[(df_final['Step'] >= max_step-num_steps) & (df_final['Step'] <= max_step)]
#     data = df_final.nlargest(num_top, "Median")["Median"]
#     return df_filt, df_final, data.mean(), data.std(), df_temp

In [6]:
path = "/Users/akashvelu/Documents/Research/icml_data/ICML_Data/"

In [7]:
def form_path(map_name, alg_name):
    return path + map_name + "/" + map_name + "_final_" + alg_name + ".csv"

In [8]:
fxn = np.median
num_top =  1
num_steps = 1e6
last_n = 10

In [9]:
easy_maps = ["2m_vs_1z", "3m", "2s_vs_1sc", "2s3z", "3s_vs_3z", "3s_Vs_4z", "so_many_baneling", "8m", "MMM", "1c3s5z", "bane_vs_bane"]
hard_maps = ["3s_vs_5z", "2c_vs_64zg", "8m_vs_9m", "25m", "5m_vs_6m", "3s5z", "10m_vs_11m"]
super_hard_maps = ["MMM2", "3s5z_vs_3s6z", "27m_vs_30m", "6h_vs_8z", "corridor"]

In [10]:
maps = easy_maps + hard_maps + super_hard_maps
difficulties = len(easy_maps) * ["Easy"] + len(hard_maps) * ["Hard"] + len(super_hard_maps) * ["Super_Hard"]
rode_scores = [np.nan, np.nan, 99.1, 98.1, np.nan, np.nan, np.nan, np.nan, np.nan, 97.2, 99.4, np.nan, 98.1, np.nan, np.nan, 80.9, 84.7, 97.1, 98.4, 95.0, 97.8, 85.6, 89.4]
rode_scores = [str(score) if score is not np.nan else "/" for score in rode_scores]

In [11]:
algs = ["mappo", "ippo", "qmix"]

In [12]:
results = np.empty((len(maps), len(algs)), dtype=object)

In [27]:
for i in range(len(maps)):
    for j in range(len(algs)):
        map_name = maps[i]
        alg_name = algs[j]
        _, val, std, all_res, err = process_data_RODE(form_path(map_name, alg_name), alg_name, fxn=fxn, last_n=last_n)
        val, std = round(val*100, 1), round(std*100, 1)
        res = str(val) + "(" + str(std) + ")"
#         if len(err) > 0:
#             if j == 2 or i > 2:
#                 cleanres = all_res[~np.isnan(all_res)]
#                 val, std = np.mean(cleanres), np.std(cleanres)
#                 val, std = round(val*100, 1), round(std*100, 1)
#                 res = str(val) + "(" + str(std) + ")"
#             else:
#                 res = "err"
        results[i][j] = res

rmappo_clean_state_xy_seed4 - eval_win_rate
Max step before:  5363200
Cut at 5e6
Max step after:  5000000.0
rmappo_clean_state_xy_seed3 - eval_win_rate
Max step before:  4963200
Cut at 5e6
Max step after:  5000000.0
rmappo_clean_state_xy_seed2 - eval_win_rate
Max step before:  4963200
Cut at 5e6
Max step after:  5000000.0
[1. 1. 1.]
1.0
rmappo_ippo_seed3 - eval_win_rate
Max step before:  9923200
Cut at 10e6
Max step after:  10000000.0
rmappo_ippo_seed2 - eval_win_rate
Max step before:  4963200
Cut at 5e6
Max step after:  5000000.0
rmappo_ippo_seed1 - eval_win_rate
Max step before:  4963200
Cut at 5e6
Max step after:  5000000.0
[1. 1. 1.]
1.0
qmix_originalstate_seed10 - eval_win_rate
Max step before:  7027720
Cut at 5e6
Max step after:  5000000.0
qmix_originalstate_seed3 - eval_win_rate
Max step before:  6167084
Cut at 5e6
Max step after:  5000000.0
qmix_originalstate_seed2 - eval_win_rate
Max step before:  6307188
Cut at 5e6
Max step after:  5000000.0
[0.96875 1.      0.96875]
0.96875


In [ ]:
maps

In [20]:
df = pandas.DataFrame({'Map': np.array(maps), "Map Difficulty": np.array(difficulties), 'MAPPO': results[:, 0], 'IPPO': results[:, 1], 'QMIX': results[:, 2], 'RODE': np.array(rode_scores)})

In [21]:
df

,Map,Map Difficulty,MAPPO,IPPO,QMIX,RODE
0,2m_vs_1z,Easy,100.0(0.0),100.0(0.0),96.9(1.9),/
1,3m,Easy,100.0(0.6),100.0(0.0),nan(nan),/
2,2s_vs_1sc,Easy,100.0(0.0),100.0(1.5),98.4(1.2),99.1
3,2s3z,Easy,100.0(0.7),100.0(0.0),96.9(1.5),98.1
4,3s_vs_3z,Easy,100.0(0.0),100.0(0.0),100.0(1.5),/
5,3s_Vs_4z,Easy,100.0(1.3),97.7(1.4),nan(nan),/
6,so_many_baneling,Easy,100.0(0.0),100.0(1.5),96.9(0.7),/
7,8m,Easy,100.0(0.0),100.0(0.7),96.9(1.5),/
8,MMM,Easy,96.9(2.6),96.9(0.0),99.2(1.4),/
9,1c3s5z,Easy,100.0(0.0),100.0(0.0),95.3(1.3),97.2


In [ ]:
table_latex = df.to_latex(index=False)

In [ ]:
text_file = open(path + "table.txt", "w")
text_file.write(table_latex)
text_file.close()

In [ ]:
print(u"\u00B1")

In [ ]:
u"\u00B1"

In [ ]:
t, val, std, all_res, _ = process_data(form_path("6h_vs_8z", "qmix"), "qmix", fxn=fxn, num_top=1)

In [ ]:
t

In [ ]:
t_filt = t[['Step', 'qmix_originalstate_gain1_seed3 - eval_win_rate']].dropna()

In [ ]:
max_step = t_filt[['Step']].max()['Step']

In [ ]:
max_step = 5e6

In [ ]:
t_final = t_filt.loc[(t_filt['Step'] >= max_step-1e6) & (t_filt['Step'] <= max_step)]

In [ ]:
t[['Step', 'qmix_originalstate_gain1_seed3 - eval_win_rate']].dropna()

In [ ]:
val = t_final.tail(last_n)['qmix_originalstate_seed7 - eval_win_rate'].median()

In [ ]:
val

In [ ]:
all_res

In [ ]:
max_step

In [ ]:
np.median(np.array([1, 2, np.nan]))